In [2]:
from selenium import webdriver
from time import sleep
import json
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
import xlrd

In [4]:
# 나중에 이거 주소를 리스트에 넣고 하나씩 반복해서 돌리면 될거같음
# 이거 저기 ms부분만 리스트에 넣고 url주소 바꾸면서 가져오면 될거같음 (ms가 위도경도임) ->이거안됨
# 용산구 주성동, 영등포구 양평동, 용상구 주성동, 종로구 교남동, 종로구 권농동, 종로구 종로 1가,2가, 종로구 필운동, 중구 소공동 중구 의지로1가
# 중구 필동 2가 3가
giyeok_list = ['37.576192,126.921263', '37.5672,126.9628', '37.559527,126.9435',
               '37.561,126.947','37.5634,126.9673','37.5727,126.9471','37.579542,126.910788',
               '37.563825,126.954525','37.5651,126.9391','37.572425,126.93255', '37.570583,126.957866'
               '37.569179,126.95908','37.557802,126.939513','37.567365,126.95868','37.564842,126.9638'
               '37.5623,126.9621','37.5614,126.9659','37.574,126.954','37.5931,126.9375','37.5872,126.9468',
               '37.482968,127.0634','37.513583,127.031375','37.49911,127.065463','37.488143,127.04505',
               '37.514792,127.055387','37.46436,127.1046','37.488856,127.104886','37.524142,127.0229',
               '37.5291,127.0236','37.499776,127.03895','37.4717,127.1114','37.487485,127.081638',
               '37.4766,127.101','37.525492,127.05235']
total_result = []
def crawling(data):
    driver = webdriver.Chrome('./chromedriver')
    for item in data:
        zuso = 'https://new.land.naver.com/rooms?ms='+item+',11&a=APT:OPST:ABYG:OBYG:GM:OR:VL:DDDGG:JWJT:SGJT:HOJT&e=RETAIL&aa=SMALLSPCRENT&ae=ONEROOM'
        driver.get(zuso)
        sleep(2)

        content = BeautifulSoup(driver.page_source, 'html.parser')
        driver.find_element_by_css_selector("#wrap > div.lnb_wrap > a:nth-child(4)").click()
        
        # 원룸/투룸 클릭 
        driver.find_element_by_css_selector("#wrap > div.lnb_wrap > a:nth-child(4)").click()
        sleep(1)

        # 혹시 리스트가 갱신이 안되는 경우를 제외하기위해 새로고침 버튼 클릭
        driver.find_element_by_css_selector("#filter > button > i ").click()  
        sleep(1)
        
        # 투룸 제외(원룸만) -> 이거 체크되어있는지 확인하고(area-pressed = true인지 false인지)
        # 체크되어있으면 클릭 안하도록 만들기
        content = BeautifulSoup(driver.page_source, 'html.parser')
        tworoom = content.find('button',{'aria-pressed':'false'}).text
        if tworoom != " 투룸":
            driver.find_element_by_css_selector("#type1").click()
            sleep(1)
             
        
        driver.find_element_by_css_selector("#list > div > div > div.list_fixed > div > a:nth-child(2)").click()
        sleep(1)

        actions = ActionChains(driver)

        # 페이지 전체를 스크롤다운 하려는게 아니라 옆에 리스트만 스크롤 다운하고 싶기 때문에 이걸 해줌
        scrollArea = driver.find_element_by_xpath('//*[@id="listContents1"]/div')
        sleep(1)

        # driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollArea)

        # actions.move_to_element(element).perform()
        last_height = driver.execute_script("arguments[0].scrollHeight;", scrollArea)

        try:
            a = driver.find_element_by_css_selector('#listContents1 > div > div > div.loader')
        except NoSuchElementException:
            a = None

        while(True):
            # 무한스크롤의 끝까지 늘려줌
            # 스크롤이 끝나기 전에는 loader이라는 div가 있는데 그게 더 이상 없을 때까지 스크롤을 내려줌  
            while a != None:
                driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollArea)
                sleep(1)
                try:
                    new_a = driver.find_element_by_css_selector('#listContents1 > div > div > div.loader')
                except NoSuchElementException:
                    
                    new_a = None
                sleep(1)    
                if new_a != a:
                    a = new_a

            new_height = driver.execute_script("arguments[0].scrollHeight;", scrollArea)
            if new_height == last_height:
                break
            last_height = new_height

        print("loading complete")
        # new_height = driver.execute_script("arguments[0].scrollTo(0, 0);", scrollArea)
        # 스크롤바 제일 위로 올라감
        driver.execute_script("arguments[0].scrollTo(0,0);", scrollArea)
        sleep(1)

        content = BeautifulSoup(driver.page_source, 'html.parser')
        
        # 건물명
        names = []
        list_name = content.select('#listContents1 > div > div > div > div > div > a.item_link > div.item_title > span.text')
        for item in list_name:
            names.append(item.text)

        num = len(names)
        print(num)

        # 도시명
        city = []
        Gu = []
        Dong = []
        place = []
        list_area = content.select('span.area.is-selected')
        for item in list_area:
            place.append(item.text)
        city.append(place[0])
        Gu.append(place[1])
        Dong.append(place[2])

        # 월세/전세
        types = []
        pp_type = content.select('#listContents1 > div > div > div > div > div > a.item_link > div.price_line > span.type')
        for item in pp_type:
            types.append(item.text)

        # 가격
        prices = []
        pp_price = content.select('#listContents1 > div > div > div > div > div > a > div.price_line > span.price')
        for item in pp_price:
            prices.append(item.text)

        # 건물 형태
        buildings = []
        pp_build = content.select('#listContents1 > div > div > div > div > div > a > div.info_area > p > strong')
        for item in pp_build:
            buildings.append(item.text)

        # 기타정보
        p_spec = []
        specs=[]
        pp_spec = content.select('#listContents1 > div > div > div > div > div > a > div.info_area > p > span')
        for item in pp_spec:
            p_spec.append(item.text)
        i = 0
        while i < len(p_spec):
            if i % 2 == 0:
                specs.append(p_spec[i])
            i += 1

        # 판매여부
        checks = []
        pp_check = content.select('#listContents1 > div > div > div > div > div > div.label_area > span > em.title')
        for item in pp_check:
            checks.append(item.text)

        # 날짜
        dates = []
        pp_date = content.select('#listContents1 > div > div > div > div > div > div.label_area > span > em.data')
        for item in pp_date:
            dates.append(item.text)

        for i in range(len(names)):
            h = []
            h.append(city[0])
            h.append(Gu[0])
            h.append(Dong[0])
            h.append(names[i])
            h.append(types[i])
            h.append(prices[i])
            h.append(buildings[i])
            h.append(specs[i])
            h.append(checks[i])
            h.append(dates[i])
            total_result.append(h)

    data = pd.DataFrame(total_result, columns = ["City", "District ", "Street", "Name", "PayType", "Price", "BuildType", "etc", "Sold", "Date"])
    data.drop_duplicates(inplace = True)
    data.to_csv('testCrawl_3.csv', mode='a', encoding='utf-8-sig')
    df = pd.read_csv('testCrawl_3.csv', mode='a', encoding='utf-8-sig')
    df.drop_duplicates(inplace = True)
    df.to_csv('testCrawl_3.csv', mode='a', encoding='utf-8-sig')


In [5]:
crawling(giyeok_list)

loading complete
75
loading complete
4
loading complete
293
loading complete
293
loading complete
6
loading complete
0
loading complete
79
loading complete
89
loading complete
1
loading complete
211
loading complete
0
loading complete
202
loading complete
6
loading complete
58
loading complete
39
loading complete
0
loading complete
46
loading complete
70
loading complete
199
loading complete
20
loading complete
1720
loading complete
313
loading complete
1599
loading complete
39
loading complete
86
loading complete
306
loading complete
0
loading complete
4873
loading complete
21
loading complete
114
loading complete
152
loading complete
20


In [ ]:
# utf-8 sig

In [ ]:
# encoding='cp949'

In [41]:
data = pd.read_csv('testCrawl_3.csv', encoding='cp949')

In [42]:
del data["Unnamed: 0"]

In [43]:
data.drop_duplicates(inplace = True)

In [44]:
# 면적당 가격 계산을 위해 공급면적만 따로 빼준다
data['공급면적'] = data.기타정보.str.split('/').str[0]
meonzeok = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
            'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
            '가', '나', '다', '라', '마', '바', '사', '아', '자', '차','카','타','파','하']
for item in meonzeok:
    data['공급면적'] = data.공급면적.str.split(item).str[0]  
# 정수형으로 변환
data['공급면적'] = pd.to_numeric(data['공급면적'])
del data['기타정보']

In [45]:
# 보증금 정보만 따로 넣기 + , 제거
bozeng_demo = []
for item in data['가격']:
    if '/' in item:
        a = item.split('/',1)[0]
        # 보증금 정보에서 , 제거 (정수형으로 변환하기 위해)
        b = a.replace(',','')
        bozeng_demo.append(b)
    else:
        bozeng_demo.append(0)
bozeng = []

In [46]:
bozeng = []
# 보증금 정보 정수로 변환
for item in bozeng_demo:
    if type(item) == str:
        if '억' in item:
            a = item.split('억',1)[0]
            b = item.split('억',1)[1]
            if b == '':
                bozeng.append(int(a)*100000000)
            else:
                bozeng.append(int(a)*100000000 + int(b)*10000)
        else:
            bozeng.append(int(item)*10000)
    else:
        bozeng.append(0)

In [33]:
print(bozeng)
print(len(bozeng))

[3000000, 0, 10000000, 2000000, 0, 5000000, 10000000, 0, 5000000, 0, 5000000, 10000000, 0, 5000000, 5000000, 2000000, 5000000, 5000000, 0, 5000000, 20000000, 5000000, 10000000, 5000000, 3000000, 5000000, 0, 0, 5000000, 5000000, 0, 10000000, 20000000, 5000000, 0, 40000000, 30000000, 10000000, 5000000, 0, 10000000, 3000000, 3000000, 30000000, 5000000, 0, 5000000, 0, 5000000, 5000000, 5000000, 0, 3000000, 5000000, 10000000, 10000000, 0, 5000000, 5000000, 3000000, 5000000, 0, 5000000, 0, 0, 5000000, 5000000, 5000000, 0, 0, 5000000, 0, 10000000, 5000000, 5000000, 10000000, 10000000, 10000000, 10000000, 3000000, 10000000, 10000000, 30000000, 10000000, 10000000, 5000000, 10000000, 10000000, 0, 10000000, 10000000, 1100000, 3000000, 0, 10000000, 10000000, 10000000, 10000000, 10000000, 10000000, 40000000, 1200000, 20000000, 10000000, 10000000, 0, 10000000, 0, 10000000, 10000000, 10000000, 10000000, 10000000, 60000000, 10000000, 0, 20000000, 10000000, 0, 10000000, 10000000, 40000000, 10000000, 10

In [31]:
print(len(data))

63405


In [47]:
data['보증금'] = bozeng

In [48]:
# 월세 정보에서 , 빼기
data['월세'] = data.가격.str.split('/').str[1]
data['월세'] = data['월세'].str.replace(',','')
# 이후 계산의 편의를 위해 0000을 붙여서 숫자로 변해줌(실제값으로 계산할 수 있게)
wolsae = []
for item in data['월세']:
    if type(item) == str:
        if '억' in item:
            idx_num = data[data['월세'] == item].index
            data = data.drop(idx_num)
        else:    
            wolsae.append(int(item)*10000)
    else:
        wolsae.append(0)

In [49]:
data['월세'] = wolsae

In [54]:
# 전세 가격 분리
zonsae_demo = []
for item in data['가격']:
    if '/' not in item:
        zonsae_demo.append(item)
    else:
        zonsae_demo.append(0)
data['전세'] = zonsae_demo
    
# 전세 정보에서 , 빼기
data['전세'] = data['전세'].str.replace(',','')
    
# 이후 계산의 편의를 위해 정수형으로 변환
zonsae = []
for item in data['전세']:
    if type(item) == str:
        if '억' in item:
            a = item.split('억',1)[0]
            b = item.split('억',1)[1]
            if b == '':
                zonsae.append(int(a)*100000000)
            elif '억' in b:
                idx_num = data[data['전세'] == item].index
                data = data.drop(idx_num)
            else:
                zonsae.append(int(a)*100000000 + int(b)*10000)
        else:
            zonsae.append(int(item)*10000)
    else:
        zonsae.append(0)

In [55]:
data['전세'] = zonsae

In [58]:
del data["가격"]

In [63]:
data.head()

,City,District,Street,Name,PayType,BuildType,Sold,Date,Area,Deposit,Monthly,Charter
0,서울시,서대문구,남가좌동,일반원룸,월세,원룸,확인,19.12.05.,13,3000000,300000,0
1,서울시,서대문구,남가좌동,일반원룸,전세,원룸,확인,19.12.05.,20,0,0,90000000
2,서울시,서대문구,남가좌동,일반원룸 Echo Vill,월세,원룸,확인,19.12.05.,35,10000000,500000,0
3,서울시,서대문구,남가좌동,DMC엘가(도시형) 1동,단기임대,아파트,확인,19.12.05.,21,2000000,750000,0
4,서울시,서대문구,남가좌동,DMC엘가(도시형) 1동,전세,아파트,확인,19.12.05.,48,0,0,240000000


In [62]:
data.columns = ["City", "District ", "Street", "Name", "PayType", "BuildType", "Sold", "Date", "Area", "Deposit", "Monthly", "Charter"]

In [64]:
data.to_csv('testClean_3.csv', mode='a', encoding='utf-8-sig') 

In [52]:
z = []
for item in data['전세']:
    z.append(item)
print(z)

[nan, '9000', nan, nan, '2억 4000', nan, nan, '9000', nan, '2억 5000', nan, nan, '2억 1000', nan, nan, nan, nan, nan, '4500', nan, nan, nan, nan, nan, nan, nan, '1억 3000', '7000', nan, nan, '1억 3000', nan, nan, nan, '5000', nan, nan, nan, nan, '1억 3000', nan, nan, nan, nan, nan, '5000', nan, '5500', nan, nan, nan, '7000', nan, nan, nan, nan, '9500', nan, nan, nan, nan, '7000', nan, '7000', '5000', nan, nan, nan, '5000', '5500', nan, '1억 1000', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, '2억 1000', nan, nan, nan, nan, '8000', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, '2억 1000', nan, '2억 1000', nan, nan, nan, nan, nan, nan, nan, '2억 500', nan, nan, '2억 1000', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, '1억 7000', '1억 8500', nan, nan, '1억 8500', '2억 1000', nan, nan, nan, nan, nan, nan, '2억 1000', nan, nan, nan, nan, '2억 1500', '2억 3000', nan, nan, '2억 1000', nan, nan, nan, nan, nan, '1억 4500', '1억 8900', nan

In [40]:
data.head()
print(len(data))

63387


In [9]:
def cleansing_data(csv_file):
    data = pd.read_csv(csv_file, encoding='utf-8 sig')
    del data["Unnamed: 0"]
    data.drop_duplicates(inplace = True)
    # 위에 컬럼명 붙여줌
    # data.columns = ['시', '구', '동', '이름', '월세/전세', '가격', '건물형태', '기타정보', '확인/판매여부', '날짜']
    # 면적당 가격 계산을 위해 공급면적만 따로 빼준다
    # 면적당 가격 계산을 위해 공급면적만 따로 빼준다
    data['Area'] = data.etc.str.split('/').str[0]
    cleanArea = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
            'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
            '가', '나', '다', '라', '마', '바', '사', '아', '자', '차','카','타','파','하']
    for item in cleanArea:
        data['Area'] = data.Area.str.split(item).str[0]  
    # 정수형으로 변환
    data['Area'] = pd.to_numeric(data['Area'])
    del data['etc']
    # 보증금 정보만 따로 넣기 + , 제거
    cleanDeposit = []
    for item in data['Price']:
        if '/' in item:
            a = item.split('/',1)[0]
            # 보증금 정보에서 , 제거 (정수형으로 변환하기 위해)
            b = a.replace(',','')
            cleanDeposit.append(b)
        else:
            cleanDeposit.append(0)
    Deposit = []
    # 보증금 정보 정수로 변환
    for item in cleanDeposit:
        if type(item) == str:
            if '억' in item:
                a = item.split('억',1)[0]
                b = item.split('억',1)[1]
                if b == '':
                    Deposit.append(int(a)*100000000)
                else:
                    Deposit.append(int(a)*100000000 + int(b)*10000)
            else:
                Deposit.append(int(item)*10000)
        else:
            Deposit.append(0)
    data['Deposit'] = Deposit
    # 월세 정보에서 , 빼기
    data['Monthly'] = data.Price.str.split('/').str[1]
    data['Monthly'] = data['Monthly'].str.replace(',','')
    # 이후 계산의 편의를 위해 0000을 붙여서 숫자로 변해줌(실제값으로 계산할 수 있게)
    cleanMonthly = []
    for item in data['Monthly']:
        if type(item) == str:
            if '억' in item:
                idx_num = data[data['Monthly'] == item].index
                data = data.drop(idx_num)
            else:    
                cleanMonthly.append(int(item)*10000)
        else:
            cleanMonthly.append(0)
    data['Monthly'] = cleanMonthly
    # 전세 가격 분리
    cleanCharter = []
    for item in data['Price']:
        if '/' not in item:
            cleanCharter.append(item)
        else:
            cleanCharter.append(0)
    data['Charter'] = cleanCharter

    # 전세 정보에서 , 빼기
    data['Charter'] = data['Charter'].str.replace(',','')

    # 이후 계산의 편의를 위해 정수형으로 변환
    Charter = []
    for item in data['Charter']:
        if type(item) == str:
            if '억' in item:
                a = item.split('억',1)[0]
                b = item.split('억',1)[1]
                if b == '':
                    Charter.append(int(a)*100000000)
                elif '억' in b:
                    idx_num = data[data['Charter'] == item].index
                    data = data.drop(idx_num)
                else:
                    Charter.append(int(a)*100000000 + int(b)*10000)
            else:
                Charter.append(int(item)*10000)
        else:
            Charter.append(0)        
    data['Charter'] = Charter
    del data["Price"]
    
    data.drop_duplicates(inplace = True)
    # 최종적으로 데이터를 저장할 장소
    data.to_csv('testClean_3.csv', mode='a', encoding='utf-8-sig') 

In [10]:
cleansing_data('testCrawl_3.csv')

ValueError: invalid literal for int() with base 10: '851억0000'

In [12]:
data = pd.read_csv('testCrawl_2.csv', encoding='utf-8 sig')
del data["Unnamed: 0"]
data.drop_duplicates(inplace = True)
# 위에 컬럼명 붙여줌
# data.columns = ['시', '구', '동', '이름', '월세/전세', '가격', '건물형태', '기타정보', '확인/판매여부', '날짜']
# 면적당 가격 계산을 위해 공급면적만 따로 빼준다
data['공급면적'] = data.기타정보.str.split('/').str[0]
meonzeok = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
            'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
            '가', '나', '다', '라', '마', '바', '사', '아', '자', '차','카','타','파','하']
for item in meonzeok:
    data['공급면적'] = data.공급면적.str.split(item).str[0]  
# 정수형으로 변환
data['공급면적'] = pd.to_numeric(data['공급면적'])
del data['기타정보']

In [13]:
data.head()

,시,구,동,이름,월세/전세,가격,건물형태,확인/판매여부,날짜,공급면적
0,서울시,서대문구,남가좌동,일반원룸,월세,300/30,원룸,확인,19.12.05.,13
1,서울시,서대문구,남가좌동,일반원룸,전세,"9,000",원룸,확인,19.12.05.,20
2,서울시,서대문구,남가좌동,일반원룸 Echo Vill,월세,"1,000/50",원룸,확인,19.12.05.,35
3,서울시,서대문구,남가좌동,DMC엘가(도시형) 1동,단기임대,200/75,아파트,확인,19.12.05.,21
4,서울시,서대문구,남가좌동,DMC엘가(도시형) 1동,전세,"2억 4,000",아파트,확인,19.12.05.,48


In [14]:
# 보증금 정보만 따로 넣기 + , 제거
bozeng_demo = []
for item in data['가격']:
    if '/' in item:
        a = item.split('/',1)[0]
        # 보증금 정보에서 , 제거 (정수형으로 변환하기 위해)
        b = a.replace(',','')
        bozeng_demo.append(b)
    else:
        bozeng_demo.append(0)
bozeng = []
for item in bozeng_demo:
    if type(item) == str:
        if '억' in item:
            a = item.replace('억','')
            b = a.replace(' ','')
            bozeng.append(b+"0000")
        else:
            bozeng.append(item+"0000")
    else:
        bozeng.append(0)
# 보증금 정보를 int형으로 변환해서 컬럼 생성
bozeng = list(map(int, bozeng))
data['보증금'] = bozeng

In [15]:
data.head()

,시,구,동,이름,월세/전세,가격,건물형태,확인/판매여부,날짜,공급면적,보증금
0,서울시,서대문구,남가좌동,일반원룸,월세,300/30,원룸,확인,19.12.05.,13,3000000
1,서울시,서대문구,남가좌동,일반원룸,전세,"9,000",원룸,확인,19.12.05.,20,0
2,서울시,서대문구,남가좌동,일반원룸 Echo Vill,월세,"1,000/50",원룸,확인,19.12.05.,35,10000000
3,서울시,서대문구,남가좌동,DMC엘가(도시형) 1동,단기임대,200/75,아파트,확인,19.12.05.,21,2000000
4,서울시,서대문구,남가좌동,DMC엘가(도시형) 1동,전세,"2억 4,000",아파트,확인,19.12.05.,48,0


In [18]:
# 월세 정보에서 , 빼기
data['월세'] = data.가격.str.split('/').str[1]
data['월세'] = data['월세'].str.replace(',','')
# 이후 계산의 편의를 위해 0000을 붙여서 숫자로 변해줌(실제값으로 계산할 수 있게)
wolsae = []
for item in data['월세']:
    if type(item) == str:
        if '억' in item:
            idx_num = data[data['월세'] == item].index
            data = data.drop(idx_num)
        else:    
            wolsae.append(item+"0000")
    else:
        wolsae.append(0)
# # int형으로 변환
# wolsae = list(map(int, wolsae))
# data['월세'] = wolsae

In [20]:
# int형으로 변환
wolsae = list(map(int, wolsae))
data['월세'] = wolsae

In [21]:
print(wolsae)

[300000, 0, 500000, 750000, 0, 420000, 570000, 0, 500000, 0, 450000, 500000, 0, 400000, 350000, 750000, 400000, 450000, 0, 400000, 550000, 250000, 350000, 500000, 300000, 250000, 0, 0, 370000, 470000, 0, 600000, 200000, 300000, 0, 50000, 100000, 500000, 400000, 0, 550000, 430000, 350000, 300000, 450000, 0, 350000, 0, 450000, 400000, 600000, 0, 380000, 400000, 350000, 570000, 0, 400000, 450000, 380000, 600000, 370000, 0, 0, 350000, 450000, 500000, 0, 0, 400000, 0, 400000, 420000, 470000, 550000, 400000, 1000000, 1000000, 300000, 600000, 500000, 600000, 600000, 450000, 800000, 650000, 0, 1000000, 950000, 900000, 800000, 0, 650000, 900000, 850000, 950000, 950000, 600000, 550000, 1200000, 570000, 680000, 800000, 0, 600000, 0, 1050000, 800000, 800000, 650000, 800000, 700000, 800000, 0, 570000, 650000, 0, 700000, 750000, 450000, 700000, 600000, 500000, 600000, 700000, 700000, 700000, 700000, 850000, 710000, 700000, 600000, 500000, 0, 0, 800000, 1000000, 0, 0, 450000, 900000, 100000, 620000, 

In [24]:
# 전세 가격 분리
zonsae_demo = []
for item in data['가격']:
    if '/' not in item:
        zonsae_demo.append(item)
    else:
        zonsae_demo.append(0)
data['전세'] = zonsae_demo
    
# 전세 정보에서 , 빼기
data['전세'] = data['전세'].str.replace(',','')
    
# 이후 계산의 편의를 위해 0000을 붙여서 숫자로 변해줌(실제값으로 계산할 수 있게)
zonsae = []
for item in data['전세']:
    if type(item) == str:
        if '억' in item:
            num = data.전세.str.split('억').str[1]
            if len(num) = 0:
                a = item.replace('억','')
                zonsae.append(a+"00000000")
            else:
                a = item.replace('억','')
                b = a.replace(' ','')
            zonsae.append(b+"0000")
        else:
            zonsae.append(item+"0000")
    else:
        zonsae.append(0)
# int형으로 변환
zonsae = list(map(int, zonsae))        
data['전세'] = zonsae

SyntaxError: invalid syntax (<ipython-input-24-79db99ddad67>, line 18)

In [42]:
zonsae = list(map(int, zonsae))        
data['전세'] = zonsae

ValueError: Length of values does not match length of index

In [26]:
# 전세 가격 분리
zonsae_demo = []
for item in data['가격']:
    if '/' not in item:
        zonsae_demo.append(item)
    else:
        zonsae_demo.append(0)
data['전세'] = zonsae_demo
    
# 전세 정보에서 , 빼기
data['전세'] = data['전세'].str.replace(',','')

In [40]:
zonsae = []
for item in data['전세']:
    if type(item) == str:
        if '억' in item:
            num = data.전세.str.split('억').str[1]
            for item in num:
                if type(num) == str:
                    if len(num) == 0:
                        a = item.replace('억','')
                        zonsae.append(a+"00000000")
                    if len(num) == 3:
                        a = item.replace('억','')
                        b = a.replace(' ','')
                        zonsae.append(b+"000000")
                    if len(num) == 4:
                        a = item.replace('억','')
                        b = a.replace(' ','')
                        zonsae.append(b+"00000")
                    if len(num) == 5:
                        a = item.replace('억','')
                        b = a.replace(' ','')
                        zonsae.append(b+"0000")
        else:
            zonsae.append(item+"0000")
    else:
        zonsae.append(0)

In [41]:
print(zonsae)


[0, '90000000', 0, 0, 0, 0, '90000000', 0, 0, 0, 0, 0, 0, 0, 0, '45000000', 0, 0, 0, 0, 0, 0, 0, '70000000', 0, 0, 0, 0, 0, '50000000', 0, 0, 0, 0, 0, 0, 0, 0, 0, '50000000', 0, '55000000', 0, 0, 0, '70000000', 0, 0, 0, 0, '95000000', 0, 0, 0, 0, 0, '70000000', '50000000', 0, 0, 0, '50000000', '55000000', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '80000000', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '50000000', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '80000000', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '89000000', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [35]:
num = data.전세.str.split('억').str[1]
zonsae = []
for item in num:
    if type(item) == str:
        if len(item) == 5:
            a = item.replace('억','')
            b = a.replace(' ','')
            zonsae.append(b+"0000")
        if len(item) == 4:
            a = item.replace('억','')
            b = a.replace(' ','')
            zonsae.append(b+"00000")
        if len(item) == 3:
            a = item.replace('억','')
            b = a.replace(' ','')
            zonsae.append(b+"000000")
        if len(item) == 0:
            a = item.replace('억','')
            zonsae.append(a+"00000")
            

 4000 5
 5000 5
 1000 5
 3000 5
 3000 5
 3000 5
 1000 5
 1000 5
 1000 5
 1000 5
 500 4
 1000 5
 7000 5
 8500 5
 8500 5
 1000 5
 1000 5
 1500 5
 3000 5
 1000 5
 4500 5
 8900 5
 3000 5
 5000 5
 1000 5
 1000 5
 6500 5
 4500 5
 5000 5
 20 3
 4000 5
 1000 5
 6800 5
 4000 5
 1000 5
 1500 5
 1000 5
 4000 5
 1000 5
 4000 5
 4500 5
 1000 5
 1000 5
 3000 5
 1000 5
 1500 5
 2000 5
 1000 5
 3000 5
 1000 5
 1000 5
 1000 5
 1000 5
 1000 5
 1000 5
 1000 5
 3000 5
 1000 5
 5000 5
 3000 5
 1000 5
 1000 5
 1500 5
 8500 5
 1000 5
 3000 5
 1000 5
 1000 5
 1000 5
 5000 5
 8500 5
 1000 5
 8000 5
 7400 5
 6000 5
 3000 5
 3000 5
 0
 3000 5
 0
 9000 5
 9000 5
 0
 5000 5
 0
 6000 5
 0
 1000 5
 6000 5
 1000 5
 2000 5
 5000 5
 6000 5
 8500 5
 3000 5
 5000 5
 7500 5
 0
 4500 5
 6000 5
 2000 5
 3000 5
 1000 5
 7500 5
 3000 5
 3000 5
 3000 5
 4000 5
 7000 5
 5000 5
 0
 4000 5
 8500 5
 8500 5
 5000 5
 0
 3000 5
 5000 5
 0
 5000 5
 6000 5
 5000 5
 6000 5
 3000 5
 1000 5
 2000 5
 2000 5
 5000 5
 3000 5
 3000 5
 5000 5


 6000 5
 5000 5
 0
 0
 2000 5
 7000 5
 8000 5
 8500 5
 0
 1000 5
 1000 5
 6000 5
 8000 5
 8000 5
 1000 5
 6000 5
 9000 5
 5000 5
 500 4
 6800 5
 5500 5
 3000 5
 5700 5
 5700 5
 2000 5
 5000 5
 5000 5
 0
 5000 5
 1000 5
 6000 5
 1000 5
 1000 5
 5000 5
 0
 2000 5
 2000 5
 0
 0
 8500 5
 5300 5
 3000 5
 7000 5
 0
 6000 5
 0
 7000 5
 2000 5
 8500 5
 0
 5000 5
 0
 6000 5
 6000 5
 7000 5
 2000 5
 2000 5
 5000 5
 2000 5
 0
 3000 5
 0
 5000 5
 5000 5
 2500 5
 6000 5
 7000 5
 7000 5
 2000 5
 2000 5
 5000 5
 2500 5
 7000 5
 0
 6000 5
 0
 1000 5
 0
 7000 5
 5000 5
 9000 5
 2000 5
 0
 2500 5
 8500 5
 0
 6600 5
 6500 5
 6500 5
 8500 5
 2000 5
 5000 5
 6000 5
 6000 5
 6000 5
 2000 5
 2000 5
 8000 5
 2000 5
 2000 5
 0
 8000 5
 5700 5
 5500 5
 6800 5
 8000 5
 500 4
 3000 5
 6000 5
 2500 5
 1000 5
 9400 5
 1800 5
 7500 5
 7000 5
 0
 8000 5
 9800 5
 7400 5
 1000 5
 3000 5
 8000 5
 5000 5
 7000 5
 2500 5
 3000 5
 5000 5
 5000 5
 7000 5
 7000 5
 6000 5
 3000 5
 2000 5
 1000 5
 7500 5
 3000 5
 8000 5
 8000 

In [ ]:
if type(item) == str:
        if len(item) == 4:
            a = item.replace('억','')
                b = a.replace(' ','')
            zonsae.append(b+"0000")
        if len(item) == 3:
            a = item.replace('억','')
                b = a.replace(' ','')
            zonsae.append(b+"0000")
            
    
print(item + ' '+ str(len(item))

In [ ]:
print

In [23]:
# 최종적으로 데이터를 저장할 장소
data.to_csv('testClean_2.csv', mode='a', encoding='utf-8-sig') 